In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import copy
import os

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class PenalizedMSELoss(nn.Module):
    def __init__(self, penalty_factor=10.0):
        super().__init__()
        self.mse = nn.MSELoss()
        self.penalty_factor = penalty_factor

    def forward(self, predictions, targets, mean=0, std=0):
        mse_loss = self.mse(predictions, targets)
        
        # Penalty for negative predictions
        penalty = torch.relu(-(predictions * std + mean)).sum()
        total_loss = mse_loss + self.penalty_factor * penalty

        return total_loss

In [ ]:
class NNC2PS(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 600)
        self.fc2 = nn.Linear(600, 200)
        self.fc3 = nn.Linear(200, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x)) 
        x = self.fc3(x)
        return x

model = NNC2PS().to(device)
model = nn.DataParallel(model)
print(f"Using GPUs: {model.device_ids}")

optimizer = optim.Adam(model.parameters(), lr=3e-4)
penalty_factor = 1.50e2
criterion = PenalizedMSELoss(penalty_factor=penalty_factor)

In [ ]:
class NNC2PL(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x)) 
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x) 
        return x

In [ ]:
torch.manual_seed(0)

n_samples = 500000
gamma_th = 5/3
K = [8.9493e-02, None, None, None]  
Gamma = [1.3569e+00, 3.0050e+00, 2.9880e+00, 2.8510e+00]
rho_breaks = [2.3674e-04, 8.1147e-04, 1.6191e-03]

rho_min = 2e-5
rho_max = 2e-3
rho = rho_min + (rho_max - rho_min) * torch.rand(n_samples, 1)
vx  = 0.721 * torch.rand(n_samples, 1)
W = (1 - vx**2).pow(-0.5)

a_i = torch.zeros(len(K))
for i in range(1, len(K)):
    K[i] = K[i-1] * rho_breaks[i-1]**(Gamma[i-1] - Gamma[i])
    a_i[i] = a_i[i-1] + K[i-1] * rho_breaks[i-1]**(Gamma[i-1] - 1) / (Gamma[i-1] - 1) - K[i] * rho_breaks[i-1]**(Gamma[i] - 1) / (Gamma[i] - 1)

p = torch.zeros_like(rho)
eps_cold = torch.zeros_like(rho)
h = torch.zeros_like(rho)
eps = torch.zeros_like(rho)
for i in range(len(K)):
    if i == 0:
        mask = rho < rho_breaks[i]
    elif i < len(K) - 1:
        mask = (rho >= rho_breaks[i-1]) & (rho < rho_breaks[i])
    else:
        mask = rho >= rho_breaks[i-1]
    
    eps_cold[mask] = a_i[i] + K[i] * rho[mask]**(Gamma[i] - 1) / (Gamma[i] - 1)
    p[mask] = K[i] * rho[mask]**Gamma[i]

eps_min = 0
eps_max = 2
eps_th = eps_min + (eps_max - eps_min) * torch.rand(n_samples, 1)
p_th = (gamma_th - 1) * rho * eps_th
p += p_th
h = 1 + eps_cold + eps_th + p / rho

D = rho * W
Sx = rho * h * W**2 * vx
tau = rho * h * W**2 - p - D

inputs = torch.cat((D, Sx, tau), dim=1)
outputs = copy.deepcopy(p)

dataset = torch.utils.data.TensorDataset(inputs, outputs)

test_size = int(0.05 * n_samples)  # 5% for testing
val_size = int(0.05 * n_samples)   # 5% for validation
train_size = n_samples - test_size - val_size  # Remaining 90% for training

# Split the dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

inputs_train_unscaled, outputs_train_unscaled = train_dataset[:]
inputs_val_unscaled, outputs_val_unscaled = val_dataset[:]
inputs_test_unscaled, outputs_test_unscaled = test_dataset[:]


In [ ]:
num_negative_outputs = torch.sum(torch.lt(outputs_train_unscaled, 0)).item()
print(f'Number of Negative Outputs: {num_negative_outputs}')

In [ ]:
log_rho = np.log10(rho)
log_p = np.log10(p)
log_eps = np.log10(eps_cold + eps_th)
log_h = np.log10(h)

# Sort indices
sort_indices = np.argsort(log_rho, axis=0).flatten()

log_rho_sorted = log_rho[sort_indices]
log_p_sorted = log_p[sort_indices]
log_eps_sorted = log_eps[sort_indices]
log_h_sorted = log_h[sort_indices]

# fs = 22
# fig, axs = plt.subplots(1, 3, figsize=(30, 10))

# axs[0].scatter(log_rho_sorted, log_p_sorted, s=1)
# for rho_break in rho_breaks:
#     axs[0].axvline(x=np.log10(rho_break), color='r', linestyle='--')
# axs[0].set_xlabel(r'$\log \rho$', fontsize=fs)
# axs[0].set_ylabel(r'$\log p$', fontsize=fs)
# axs[0].set_title(r'$\log \rho$ vs $\log p$', fontsize=fs)
# axs[0].grid(True)

# axs[1].scatter(log_rho_sorted, log_eps_sorted, s=1)
# for rho_break in rho_breaks:
#     axs[1].axvline(x=np.log10(rho_break), color='r', linestyle='--')
# axs[1].set_xlabel(r'$\log \rho$', fontsize=fs)
# axs[1].set_ylabel(r'$\log \epsilon$', fontsize=fs)
# axs[1].set_title(r'$\log \rho$ vs $\log \epsilon$', fontsize=fs)
# axs[1].grid(True)

# axs[2].scatter(log_rho_sorted, log_h_sorted, s=1)
# for rho_break in rho_breaks:
#     axs[2].axvline(x=np.log10(rho_break), color='r', linestyle='--')
# axs[2].set_xlabel(r'$\log \rho$', fontsize=fs)
# axs[2].set_ylabel(r'$\log h$', fontsize=fs)
# axs[2].set_title(r'$\log \rho$ vs $\log h$', fontsize=fs)
# axs[2].grid(True)

# plt.tight_layout()
# plt.savefig('../images/piecewise_data_mpl.png', dpi=600)
# plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def create_publication_plots(log_rho_sorted, log_p_sorted, log_eps_sorted, log_h_sorted, rho_breaks):
    """
    Create publication-quality plots using seaborn for EOS-related data.
    """
    # Set the style for publication-quality plots
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=2)
    
    # Create figure with three subplots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    
    # Common plotting parameters
    plot_params = dict(
        marker='o',
        s=10,  # Larger point size for better visibility
        alpha=0.6,  # Some transparency
        color=sns.color_palette("deep")[0],  # Use seaborn's color palette
        linewidth=0
    )
    
    # Common axis parameters
    def setup_axis(ax, x_label, y_label, title):
        ax.tick_params(labelsize=18)
        ax.set_xlabel(x_label, fontsize=24, fontweight='bold')
        ax.set_ylabel(y_label, fontsize=24, fontweight='bold')
        # ax.set_title(title, fontsize=24, pad=20)
        
        # Add vertical lines for density breaks
        for rho_break in rho_breaks:
            ax.axvline(x=np.log10(rho_break), color='red', 
                      linestyle='--', linewidth=2, alpha=0.7)
    
    # First subplot: log ρ vs log p
    ax1.scatter(log_rho_sorted, log_p_sorted, **plot_params)
    setup_axis(ax1, r'$\log \rho$', r'$\log p$', r'$\log \rho$ vs $\log p$')
    
    # Second subplot: log ρ vs log ε
    ax2.scatter(log_rho_sorted, log_eps_sorted, **plot_params)
    setup_axis(ax2, r'$\log \rho$', r'$\log \epsilon$', r'$\log \rho$ vs $\log \epsilon$')
    
    # Third subplot: log ρ vs log h
    ax3.scatter(log_rho_sorted, log_h_sorted, **plot_params)
    setup_axis(ax3, r'$\log \rho$', r'$\log h$', r'$\log \rho$ vs $\log h$')
    
    # Adjust layout
    plt.tight_layout()
    
    # Save figure
    plt.savefig('../images/piecewise_data.png', bbox_inches='tight', dpi=600)
    
    return fig, (ax1, ax2, ax3)

# Example usage:
fig, axes = create_publication_plots(log_rho_sorted, log_p_sorted, 
                                     log_eps_sorted, log_h_sorted, rho_breaks)
plt.show()

In [ ]:
input_scaler = StandardScaler()
inputs_train = torch.from_numpy(input_scaler.fit_transform(inputs_train_unscaled))
inputs_val = torch.from_numpy(input_scaler.transform(inputs_val_unscaled))
inputs_test = torch.from_numpy(input_scaler.transform(inputs_test_unscaled))

output_scaler = StandardScaler()
outputs_train = torch.from_numpy(output_scaler.fit_transform(outputs_train_unscaled))
outputs_val = torch.from_numpy(output_scaler.fit_transform(outputs_val_unscaled))
outputs_test = torch.from_numpy(output_scaler.transform(outputs_test_unscaled))

train_dataset = TensorDataset(inputs_train, outputs_train)
val_dataset = TensorDataset(inputs_val, outputs_val)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
os.makedirs('checkpoints', exist_ok=True)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
n_epochs = 85 # 85 seems to be the ideal epoch for this model

train_mean = torch.mean(outputs_train_unscaled, dim=0).to(device)
train_std = torch.std(outputs_train_unscaled, unbiased=False).to(device)

train_losses = []
val_losses = []

# Function to get the current learning rate
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# Training loop
for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    for batch_inputs, batch_outputs in train_loader:
        batch_inputs = batch_inputs.float()
        batch_outputs = batch_outputs.float()
        
        batch_inputs = batch_inputs.to(device)
        batch_outputs = batch_outputs.to(device)
        
        optimizer.zero_grad()
        batch_outputs_pred = model(batch_inputs)
        
        loss = criterion(batch_outputs_pred, batch_outputs, train_mean, train_std)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_outputs in val_loader:
            batch_inputs = batch_inputs.float()
            batch_outputs = batch_outputs.float()
            
            batch_inputs = batch_inputs.to(device)
            batch_outputs = batch_outputs.to(device)
            
            batch_outputs_pred = model(batch_inputs)
            loss = criterion(batch_outputs_pred, batch_outputs, train_mean, train_std)
            
            val_loss += loss.item() * batch_inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    
    current_lr = get_lr(optimizer)
    print(f'Epoch {epoch+1}/{n_epochs} Train Loss: {train_loss:.16f} Val Loss: {val_loss:.16f} LR: {current_lr:.6f}')
    
    scheduler.step(val_loss)

    # Check GPU memory allocation
    for i in range(torch.cuda.device_count()):
        print(f"Memory allocated on GPU {i}: {torch.cuda.memory_allocated(i)} bytes")
    
    # Save checkpoint every 100 epochs
    if (epoch + 1) % 200 == 0:
        checkpoint_path = f'checkpoints/checkpoint_epoch_{epoch+1}.pth'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
        }, checkpoint_path)
        print(f'Checkpoint saved at {checkpoint_path}')


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.semilogy(range(1, n_epochs + 1), train_losses, label='Training Loss')
plt.semilogy(range(1, n_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss vs. Epochs')
plt.show()

In [ ]:
(np.argmin(val_losses), np.min(val_losses))

In [ ]:
if isinstance(model, torch.nn.DataParallel):
    model = model.module
    
model = model.to('cpu')

In [ ]:
def inverse_standard_scaler(standardized_tensor, mean, std):
    # Inverse standardize the tensor
    original_tensor = (standardized_tensor * std) + mean
    
    return original_tensor

In [ ]:
# Ensure input and output tensors are on CPU
inputs_test = inputs_test.to('cpu')
outputs_test = outputs_test.to('cpu')

model.eval()

# Perform inference
with torch.no_grad():
    predictions = model(inputs_test.float())
    print(predictions[:10])
inverted_predictions = inverse_standard_scaler(predictions, torch.mean(outputs_train_unscaled, dim=0), torch.std(outputs_train_unscaled, unbiased=False))

# L1 Loss
l1_loss = nn.L1Loss()
l1_error = l1_loss(inverted_predictions, outputs_test_unscaled)
print(f'L1 Error: {l1_error.item():.2e}')

# Mean Squared Error (MSE) and L2 Loss
mse_loss = nn.MSELoss()
error = mse_loss(inverted_predictions, outputs_test_unscaled)
l2_error = torch.sqrt(error)
print(f'L2 Error: {l2_error.item():.2e}')

# L-infinity norm (max absolute error)
linf_error = torch.max(torch.abs(inverted_predictions - outputs_test_unscaled))
print(f'L-infinity Error: {linf_error.item():.2e}')

In [ ]:
inverted_predictions  # Check the length of the test outputs

In [ ]:
outputs_test_unscaled

In [ ]:
# Invert the predictions and make sure everything is physically meaningful
# inverted_predictions = inverse_standard_scaler(predictions, torch.mean(outputs_test, dim=0), torch.std(outputs_test, unbiased=False))
print(torch.sum(torch.lt(outputs_test, 0)).item())
print(torch.sum(torch.lt(inverted_predictions, 0)).item())
print(len([ip for ip in inverted_predictions if ip < 0]))

In [ ]:
# Compute mean and standard deviation
train_mean = train_mean.to("cpu")
train_std = train_std.to("cpu")

train_mean_out_np = train_mean.numpy()
train_std_out_np = train_std.numpy()

np.savetxt("./speed_test/gpu/mean_std_out_s_pen100.txt", np.vstack((train_mean_out_np, train_std_out_np)), fmt="%.17f")

inputs_train_unscaled = inputs_train_unscaled.to("cpu")

mean = inputs_train_unscaled.mean(dim=0, keepdim=True)
std = inputs_train_unscaled.std(dim=0, keepdim=True)

train_mean_in_np = mean.numpy()
train_std_in_np = std.numpy()
np.savetxt("./speed_test/gpu/mean_std_in_s_pen100.txt", np.vstack((train_mean_in_np, train_std_in_np)), fmt="%.17f")

In [ ]:
# model_path = os.path.join("..", "models", "piecewise_polytrope_nnc2pxl_cpu.pth") # for CPU
model_path = os.getcwd() + "/piecewise_polytrope_nnc2ps_pen100_normalized_gpu.pth" # for GPU

model.eval()
model.to(device)
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, model_path)

In [ ]:
inputs_test_unscaled_np = inputs_test_unscaled.cpu().numpy()
outputs_test_unscaled_np = outputs_test_unscaled.cpu().numpy()
inputs_test_np = inputs_test.cpu().numpy()
outputs_test_np = outputs_test.cpu().numpy()
preds_test_np = predictions.cpu().numpy()
inverted_preds_np = inverted_predictions.cpu().numpy()
inputs_train_unscaled_np = inputs_train_unscaled.cpu().numpy()
outputs_train_unscaled_np = outputs_train_unscaled.cpu().numpy()

# Save to txt with maximum precision
np.savetxt('./speed_test/gpu/inputs_train_unscaled_s_pen100.txt', inputs_train_unscaled_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/outputs_train_unscaled_s_pen100.txt', outputs_train_unscaled_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/inputs_test_unscaled_s_pen100.txt', inputs_test_unscaled_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/outputs_test_unscaled_s_pen100.txt', outputs_test_unscaled_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/preds_test_s_pen100.txt', preds_test_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/inputs_test_scaled_s_pen100.txt', inputs_test_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/outputs_test_scaled_s_pen100.txt', outputs_test_np, fmt='%.9g')
np.savetxt('./speed_test/gpu/inverted_preds_s_pen100.txt', inverted_preds_np, fmt='%.9g')